In [118]:
import pandas as pd
import sys, os
import tensorflow as tf
import shape_helper
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [119]:

#the following bits are basically just to deal with translations and rotations in a pretty efficient manner. Note the 
#translation+removal functions have no for loops so it's pretty quick 
#Let's make the goal shape into a bunch of 1's and 0's representing where we want pieces
def mattoones(goalshape1):
    for i in range(len(goalshape1)):
        for k in range(len(np.transpose(goalshape1))):
            if goalshape1[i][k]!='0.0' and goalshape1[i][k]!=0:
                goalshape1[i][k]=1
    return goalshape1
#Remove all rows/columns at edges with only 0's in order to standardize calculations 
def translator(matrix):
    sums=0;
    while(sum(matrix[0])==0):
        if (sum(matrix[0]==0)):
            matrix=np.delete(matrix,0,0)

    i=len(matrix)-1
    while(sums==0):
        sums=sum(matrix[i]);
        if(sums==0):
            matrix=np.delete(matrix,i,0)
        i=i-1
    return(matrix)

#(*use the above function but for columns too*)
def removal(matrixer):
    c=translator(matrixer);
    c=translator(np.transpose(c))
    return(np.transpose(c))
def sqdiff(m1,m2):
    return (sum(sum((m1-m2)*(m1-m2))))
#I think this lets us visualize chains as 1/0s instead of HP but it's not very useful 
def graphicchainno(chain):
    boundary=np.floor(np.sqrt(len(chain))*2);
    lattice=np.ndarray.tolist(np.zeros([int(boundary),int(boundary)]))
    for i in range(len(chain)):
        lattice[chain[i][0]][chain[i][1]]=1
    lattice=np.array(lattice)
    return(lattice)

In [120]:
x=1
model=keras.Sequential([
    #Entry block: create four masks, and run
    layers.Conv2D(filters=128,kernel_size=3,activation="relu",padding='same',input_shape=[15,15,1]),
    layers.MaxPool2D(),
    layers.Dropout(.3),
    #Block 2
    layers.Conv2D(filters=128, kernel_size=3,activation="relu",padding='same'),
    layers.MaxPool2D(),layers.Dropout(.3),

    #Third hidden layer
    layers.Conv2D(filters=128,kernel_size=3,activation="relu",padding='same'),
    #layers.LSTM(25,input_shape=[13,13,1]),
    #output 
    layers.Flatten(),
    
    #layers.Reshape([1152,1]),
    layers.Dense(units=169,activation="relu"),
    #layers.Dense(units=169,activation="sigmoid")
])
#model.add(keras.layers.TimeDistributed(model))
#model.add(layers.LSTM(16))

#model.add(layers.Dense(units=48,activation="relu"))
#model.add(layers.Dense(units=16,activation="softmax"))
#model.add(layers.Reshape([16,3]))
#model.add(layers.Dense(units=169,activation="sigmoid"))
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_42 (Conv2D)          (None, 15, 15, 128)       1280      
                                                                 
 max_pooling2d_28 (MaxPoolin  (None, 7, 7, 128)        0         
 g2D)                                                            
                                                                 
 dropout_28 (Dropout)        (None, 7, 7, 128)         0         
                                                                 
 conv2d_43 (Conv2D)          (None, 7, 7, 128)         147584    
                                                                 
 max_pooling2d_29 (MaxPoolin  (None, 3, 3, 128)        0         
 g2D)                                                            
                                                                 
 dropout_29 (Dropout)        (None, 3, 3, 128)       

1+1

In [121]:
#set datasets 
#from sklearn.model_selection import train_test_split
import random
import shape_helper
df=pd.read_csv('/Users/joshuatsai/Documents/GitHub/SigmaFold/library/sequence_training.csv')
targets=df['path']
startshape=df['shape_mapping']





In [122]:
df=pd.read_csv('/Users/joshuatsai/Documents/GitHub/SigmaFold/library/sequence_training.csv')
seq=df['sequence']
length=16;
trainx=[]
for i in range(len(seq)):


    seq[i]=seq[i].replace("P","1")
    seq[i]=seq[i].replace("H","2")
    seq[i]=seq[i].ljust(length,'0')
onehotencode=np.zeros([len(seq),length,3])
for i in range(len(seq)):
    for j in range(len(seq[1])):

        onehotencode[i][j][int(seq[i][j])]=1
starts=df['path']   
arraysize=13
for i in range(len(seq)):
    c=shape_helper.deserialize_path(starts[i])
    d=path_to_shape(c,arraysize)
    trainx.append(d[0])
#seq = [f"{x:0{length}}" for x in seq]
validshapes=[]
validseqs=[]
for i in range(len(trainx)):
    prob=random.random()
    eve=len(trainx)
    if prob<.05:

        validshapes.append(trainx[eve-i])
        validseqs.append(onehotencode[eve-i])
        trainx=np.delete(trainx,eve-i,0)
        onehotencode=np.delete(onehotencode,eve-i,0)

    


/var/folders/jg/j_dsxfn90sd7szymmw_rmbnm0000gn/T/ipykernel_39735/1572312923.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seq[i]=seq[i].replace("P","1")
/var/folders/jg/j_dsxfn90sd7szymmw_rmbnm0000gn/T/ipykernel_39735/1572312923.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seq[i]=seq[i].replace("H","2")
/var/folders/jg/j_dsxfn90sd7szymmw_rmbnm0000gn/T/ipykernel_39735/1572312923.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a

startpos ==> (1, -1)
startpos ==> (1, 2)
startpos ==> (1, -1)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (-1, 1)
startpos ==> (-1, 0)
startpos ==> (-1, 0)
startpos ==> (-1, 0)
startpos ==> (0, 3)
startpos ==> (0, -2)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (-1, 0)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (-1, 1)
startpos ==> (-1, 0)
startpos ==> (-1, 0)
startpos ==> (-1, 0)
startpos ==> (0, 3)
startpos ==> (0, -2)
startpos ==> (0, 3)
startpos ==> (0, -2)
startpos ==> (0, 3)
startpos ==> (0, -2)
startpos ==> (-1, 1)
startpos ==> (-1, 0)
startpos ==> (0, 3)
startpos ==> (0, -2)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (1, 1)
startpos ==> (1, 0)
startpos ==> (1, 1)
startpos ==> (1, 0)
startpos ==> (1, 1)
startpos ==> (1, 0)
startpos ==> (1, 1)
startpos ==> (1, 0)
startp

In [123]:
df=pd.read_csv('/Users/joshuatsai/Documents/GitHub/SigmaFold/library/sequence_training.csv')
seq=df['sequence']
length=16;
trainx=[]
for i in range(len(seq)):


    seq[i]=seq[i].replace("P","1")
    seq[i]=seq[i].replace("H","2")
    seq[i]=seq[i].ljust(length,'0')
onehotencode=np.zeros([len(seq),length])
for i in range(len(seq)):
    for j in range(len(seq[1])):

        onehotencode[i][j]=int(seq[i][j])
starts=df['path']   
arraysize=13
for i in range(len(seq)):
    c=shape_helper.deserialize_path(starts[i])
    d=path_to_shape(c,arraysize)
    trainx.append(d[0])
#seq = [f"{x:0{length}}" for x in seq]
validshapes=[]
validseqs=[]
for i in range(len(trainx)):
    prob=random.random()
    eve=len(trainx)
    if prob<.05:

        validshapes.append(trainx[eve-i])
        validseqs.append(onehotencode[eve-i])
        trainx=np.delete(trainx,eve-i,0)
        onehotencode=np.delete(onehotencode,eve-i,0)

    


/var/folders/jg/j_dsxfn90sd7szymmw_rmbnm0000gn/T/ipykernel_39735/1294949533.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seq[i]=seq[i].replace("P","1")
/var/folders/jg/j_dsxfn90sd7szymmw_rmbnm0000gn/T/ipykernel_39735/1294949533.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seq[i]=seq[i].replace("H","2")
/var/folders/jg/j_dsxfn90sd7szymmw_rmbnm0000gn/T/ipykernel_39735/1294949533.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a

startpos ==> (1, -1)
startpos ==> (1, 2)
startpos ==> (1, -1)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (-1, 1)
startpos ==> (-1, 0)
startpos ==> (-1, 0)
startpos ==> (-1, 0)
startpos ==> (0, 3)
startpos ==> (0, -2)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (-1, 0)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (-1, 1)
startpos ==> (-1, 0)
startpos ==> (-1, 0)
startpos ==> (-1, 0)
startpos ==> (0, 3)
startpos ==> (0, -2)
startpos ==> (0, 3)
startpos ==> (0, -2)
startpos ==> (0, 3)
startpos ==> (0, -2)
startpos ==> (-1, 1)
startpos ==> (-1, 0)
startpos ==> (0, 3)
startpos ==> (0, -2)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (1, 1)
startpos ==> (1, 0)
startpos ==> (1, 1)
startpos ==> (1, 0)
startpos ==> (1, 1)
startpos ==> (1, 0)
startpos ==> (1, 1)
startpos ==> (1, 0)
startp

In [124]:
import math as math
def path_to_shape(path,grid_sizes):
    grid = np.asarray([[0]*grid_sizes]*grid_sizes, dtype=int) # actual grid for mapping
    temp_grid = np.asarray([[0]*grid_sizes]*grid_sizes, dtype=int) # temp grid to hold the array before alignment
    for i in range(len(path)):
        temp_grid[path[i][1]+round(grid_sizes/2)][path[i][0]+ round(grid_sizes/2)] = i+1
       # if coord==path[len(path)-1]:
            #temp_grid[coord[1]+round(grid_sizes/2)][coord[0]+ round(grid_sizes/2)] = i
            # path transferred onto grid but uncentered
    #temp_grid[round(grid_sizes/2)][round(grid_sizes/2)] = 2  

    temp_grid = np.pad(temp_grid, 1) # zero padding to avoid multiplying by 0 when calculating moments

    # find centroid of temp_grid
    m_00 = len(path) # non-zero residues
    m_01 = 0
    for row_n, row in enumerate(temp_grid):
        if np.any(row!=0):
            m_01 += row_n * np.count_nonzero(row)
    m_10 = 0
    for col_n in range(np.shape(temp_grid)[1]):
        if np.any(temp_grid[:, col_n]!=0):
            m_10 += col_n *  np.count_nonzero(temp_grid[:, col_n])

    # coordinates of centroid
    n_centroid = math.floor((m_10/m_00))
    m_centroid = math.floor((m_01/m_00))
    centroid = (m_centroid, n_centroid)
    #print("centroid ==>", centroid)
    # align temp_grid onto grid
    dev_m = round(grid_sizes/2)-centroid[0]
    dev_n = round(grid_sizes/2)-centroid[1]
    coord_list = np.nonzero(temp_grid)
    print("startpos ==>", (dev_m+1,dev_n+1))
    x,y=np.where(temp_grid==2)
    start= (x+dev_m,y+dev_n)
    x,y=np.where(temp_grid==3)
    end=(x+dev_m,y+dev_n)
    
    for i in range(len(coord_list[0])):
        grid[coord_list[0][i]+dev_m][coord_list[1][i]+dev_n] = i
        #if i==0:
            #grid[coord_list[0][i]+dev_m][coord_list[1][i]+dev_n] = 2
    
    return temp_grid, grid,start,end
def start_to_class(coords,grid_size):
    return coords[0]*grid_size+coords[1]

In [125]:
onehotencode

array([[1., 2., 2., ..., 2., 1., 2.],
       [1., 2., 2., ..., 2., 1., 2.],
       [1., 2., 2., ..., 2., 1., 2.],
       ...,
       [2., 1., 1., ..., 0., 0., 0.],
       [2., 1., 1., ..., 0., 0., 0.],
       [2., 1., 1., ..., 0., 0., 0.]])

In [126]:
from tensorflow import keras
from tensorflow.keras import layers
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=10000,
    decay_rate=0.9)
cost=
model.compile(
    optimizer=tf.keras.optimizers.Adam(epsilon=.01),
    loss='CategoricalCrossentropy',metrics=['AUC']
)


SyntaxError: invalid syntax (3519330171.py, line 7)

In [127]:
model.compile(
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.01),
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy'])

In [128]:
np.shape(onehotencode)

(5961, 16)

In [116]:
history = model.fit(
    trainx,onehotencode,epochs=20
)


Epoch 1/20


ValueError: in user code:

    File "/Users/joshuatsai/anaconda3/envs/tensorflow/lib/python3.10/site-packages/keras/engine/training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "/Users/joshuatsai/anaconda3/envs/tensorflow/lib/python3.10/site-packages/keras/engine/training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/joshuatsai/anaconda3/envs/tensorflow/lib/python3.10/site-packages/keras/engine/training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "/Users/joshuatsai/anaconda3/envs/tensorflow/lib/python3.10/site-packages/keras/engine/training.py", line 998, in train_step
        return self.compute_metrics(x, y, y_pred, sample_weight)
    File "/Users/joshuatsai/anaconda3/envs/tensorflow/lib/python3.10/site-packages/keras/engine/training.py", line 1092, in compute_metrics
        self.compiled_metrics.update_state(y, y_pred, s

In [51]:
np.shape(validshapes)
validate=tf.convert_to_tensor(validshapes)

In [52]:
xk=model.predict(validate)

10/10 [==============================] - 1s 32ms/step


In [53]:
validseqs[1]

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]])

In [41]:
xk[1]

array([[ 0.11947478,  0.4727159 ,  0.4835624 ],
       [-0.04792286,  0.28662995,  0.66146725],
       [ 0.14783874,  0.44676772,  0.3480342 ],
       [-0.11151874,  0.5052162 ,  0.39022943],
       [-0.01733312,  0.38831505,  0.5220802 ],
       [-0.02375326,  0.52662975,  0.3332652 ],
       [ 0.09767437,  0.4986537 ,  0.45183194],
       [-0.09120099,  0.4311516 ,  0.30558285],
       [ 0.01774469,  0.2489044 ,  0.474659  ],
       [-0.02282003,  0.57187456,  0.44718784],
       [-0.02668642,  0.3769759 ,  0.5541784 ],
       [ 0.03583835,  0.5339102 ,  0.60561943],
       [-0.02958344,  0.29598337,  0.41882813],
       [ 0.2985186 ,  0.34509695,  0.2717138 ],
       [ 0.35686263,  0.14888875,  0.36154088],
       [ 0.63117886, -0.02104348,  0.13570109]], dtype=float32)

In [67]:
validseqs


[array([[0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.]]),
 array([[0., 1., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.]]),
 array([[0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [1., 0., 0.],
      

In [73]:
acc(c,validseqs)/2

2269.0

In [72]:
len(c)*16

4816

In [70]:
def acc(array1,array2):
    total=0
    for i in range(len(array1)):
        for j in range(len(array1[1])):
                for k in range(len(array1[1][1])):
                    if array1[i][j][k]!=array2[i][j][k]:
            
                        total=1+total
    return total
def threshold(array1):
    for i in range(len(array1)):
        for j in range(len(array1[1])):
            array1[i][j]=round(array1[i][j])
    return array1
def accen(array1,array2):
    total=0
    for i in range(len(array1)):
        for j in range(len(array1[1])):
                if array1[i][j]!=array2[i][j]:
            
                    total=1+total
    return total
def thresholden(array1):
    for i in range(len(array1)):
        for j in range(len(array1[1])):
            array1[i][j][np.where(array1[i][j]==max(array1[i][j]))]=1

            array1[i][j][np.where(array1[i][j]!=max(array1[i][j]))]=0
        
    return array1



In [ ]:
trues=np.zeros([13,13])
for i in range(len(thresx)):
    if len(np.where(thresx[i]==1)[0])==1:
        trues[math.floor(np.where(valy[i]==1)[0][0]/13)][np.where(valy[i]==1)[0][0]%13]=trues[math.floor(np.where(valy[i]==1)[0][0]/13)][np.where(valy[i]==1)[0][0]%13]+1
    #if i<len(thresx)-1:
        #if(np.where(trainy[i]==1)==np.where(thresx[i+1]==1)):
           # print(i)
            #cor=cor+1
    #if i>0:
        #if(np.where(trainy[i]==1)==np.where(thresx[i-1]==1)):
            #print(i)
            #cor=cor+1
zs=np.zeros([13,13])
for i in range(len(thresx)):
    if len(np.where(thresx[i]==1)[0])==1:
        zs[math.floor(np.where(thresx[i]==1)[0][0]/13)][np.where(thresx[i]==1)[0][0]%13]=zs[math.floor(np.where(thresx[i]==1)[0][0]/13)][np.where(thresx[i]==1)[0][0]%13]+1
    #if i<len(thresx)-1:
        #if(np.where(trainy[i]==1)==np.where(thresx[i+1]==1)):
           # print(i)
            #cor=cor+1
    #if i>0:
        #if(np.where(trainy[i]==1)==np.where(thresx[i-1]==1)):
            #print(i)
            #cor=cor+1


In [ ]:
#Our final accuracy statement
correct/len(valy)


In [ ]:
print(trues)
print(zs)

In [ ]:
cor=0
for i in range(len(thresx)):
    if(np.where(valy[i]==1)==np.where(thresx[i]==1)):
        print(i)
        cor=cor+1
    #if i<len(thresx)-1:
        #if(np.where(trainy[i]==1)==np.where(thresx[i+1]==1)):
           # print(i)
            #cor=cor+1
    #if i>0:
        #if(np.where(trainy[i]==1)==np.where(thresx[i-1]==1)):
            #print(i)
            #cor=cor+1
cor

In [ ]:
for i in range(len(thresx)):
    
    print(np.where(thresx[i]==1))

In [ ]:
np.where(thresx[5]==1)[0]